[] 3.1 Use gensim.downloader.load to load the word2vec-google-news-300 pretrained embedding model.

In [1]:
import gensim.downloader

word2vec_googlenews = gensim.downloader.load('word2vec-google-news-300')



3.2 Use the tokenizer from nltk to extract words from the Reddit posts. Display the number of tokens in the training set.


In [2]:
import json
import gzip
import nltk

jsonfiledirectory = "/Users/rosiers/Documents/GitHub/472-assignment1/goemotions.json.gz"

with gzip.open(jsonfiledirectory, "r") as f:
    data = json.loads(f.read().decode("utf-8"))
strings_token = []
for item in data:
    strings_token.append(nltk.word_tokenize(item[0]))
count = 0
for str in strings_token:
    count = count + len(str)
print(count)

2642128


3.3 Compute the embedding of a Reddit post as the average of the embeddings of its words. If a word has no embedding in Word2Vec, skip it.

In [3]:
# create an empty list to add each average embedding to
embeddings = []
#strings_token is the tokenized sentences
for s in strings_token:
    #for each sentence create a vector to hold the embeddings for each token in the string
    # it will be wiped each time
    strings_embedding = []

    #for each token in the sentence check if it has embedding and add it to the list or skip
    for tkn in s:
        if tkn in word2vec_googlenews:
            strings_embedding.append(word2vec_googlenews[tkn])
        else:
            continue

    #use zip to put all elements from each embedding into list of the same index
    embd = zip(*strings_embedding)
    #calculate the average by finding the sum of the elements at each index, and dividing it by the number of elements
    average_embedding = []
    for e in embd:
        avg = sum(e)/len(e)
        average_embedding.append(avg)
    #once all the averages are computed, append the average embedding as an element
    embeddings.append(average_embedding)
print(embeddings[1])

[-0.03008478338068182, 0.09322903373024681, 0.030028256503018467, 0.07418684525923296, -0.11172897165471857, -0.04322398792613636, 0.03311573375355114, -0.0864347978071733, 0.03924486853859641, 0.06014737215909091, -0.012917258522727272, -0.15264060280539773, 0.002850619229403409, 0.010581276633522728, -0.13426659323952414, 0.09747314453125, 0.04696932705965909, 0.14509721235795456, -0.020958640358664772, -0.10691972212357954, 0.009561018510298296, 0.07356678355823863, 0.13306149569424716, -0.02817327325994318, 0.10091330788352272, -0.05473605069247159, -0.048980712890625, 0.05866172096946023, 0.012480302290482954, -0.005160245028409091, -0.02615148370916193, 0.03894979303533381, -0.022986672141335228, -0.020204023881392044, 0.08096036044034091, -0.052642822265625, 0.008706526322798296, -0.0028459375554865055, 0.05139090798117898, 0.07250282981178978, 0.14099675958806818, -0.051161332563920456, 0.12253639914772728, -0.016482960094105114, -0.0491301796653054, -0.04001548073508523, -0.06

3.4 Compute and display the overall hit rates of the training and test sets (i.e. the % of words in the Reddit posts for which an embedding is found in Word2Vec).

In [9]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
import numpy

posts = []
sentiment = []
emotions = []
for item in data:
    posts.append(item[0])
    sentiment.append(item[2])
    emotions.append(item[1])

X_train, X_test, y_train, y_test = train_test_split(embeddings, sentiment, test_size=0.2)

print(X_train[0])
print(y_train[0])
#x_numpy = numpy.array(X_train).reshape(-1,1)
#y_numpy = numpy.array(y_train).reshape(-1,1)
basemlp = MLPClassifier(max_iter=4)
basemlp.fit(X_train, y_train)
print(basemlp.score(X_test,y_test))

[-0.03008478338068182, 0.09322903373024681, 0.030028256503018467, 0.07418684525923296, -0.11172897165471857, -0.04322398792613636, 0.03311573375355114, -0.0864347978071733, 0.03924486853859641, 0.06014737215909091, -0.012917258522727272, -0.15264060280539773, 0.002850619229403409, 0.010581276633522728, -0.13426659323952414, 0.09747314453125, 0.04696932705965909, 0.14509721235795456, -0.020958640358664772, -0.10691972212357954, 0.009561018510298296, 0.07356678355823863, 0.13306149569424716, -0.02817327325994318, 0.10091330788352272, -0.05473605069247159, -0.048980712890625, 0.05866172096946023, 0.012480302290482954, -0.005160245028409091, -0.02615148370916193, 0.03894979303533381, -0.022986672141335228, -0.020204023881392044, 0.08096036044034091, -0.052642822265625, 0.008706526322798296, -0.0028459375554865055, 0.05139090798117898, 0.07250282981178978, 0.14099675958806818, -0.051161332563920456, 0.12253639914772728, -0.016482960094105114, -0.0491301796653054, -0.04001548073508523, -0.06

/Users/rosiers/Library/jupyterlab-desktop/jlab_server/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (4) reached and the optimization hasn't converged yet.
  warnings.warn(


0.34457571877546267


3.5 Train a Base-MLP: a Multi-Layered Perceptron (neural network.MLPClassifier) with the default parameters.

3.6 Train a Top-MLP: a better performing Multi-Layered Perceptron found with whatever hyperparameters you want.

In [10]:
from sklearn.model_selection import GridSearchCV
topmlp = MLPClassifier(max_iter=1)

param_grid = {
    'activation' : ['sigmoid', 'tanh', 'relu', 'identity'],
    'hidden_layer_sizes': [(10,10,10), (30,50)],
    'solver': ['sgd', 'adam']
}
mlp_grid = GridSearchCV(topmlp,param_grid,n_jobs=-1)
mlp_grid.fit(X_train, y_train)
print("TOP-MLP: ")
print(mlp_grid.score(X_test, y_test))


/Users/rosiers/Library/jupyterlab-desktop/jlab_server/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


KeyboardInterrupt: 

3.7  Display the performance of your classifiers using metrics.classification report and add these to your performance file.